# Métodos de Classificação
#### Para alterar o valor que se deseja classificar, altere o valor das variaveis COLUMN e  VALUE (na quarta célula). Esses valores são definidos com base em como o pandas adiciona colunas no dataframe quando a função get_dummies é chamada. Por exemplo, para uma coluna chamada "Concedido", que possui apenas valores "S" ou "N", duas novas colunas são criadas (após a chamada de get_dummies), a saber: Concedido_S e Condedido_N. Dessa forma, pode-se ter COLUMN = 'Concedido' e VALUE = 'S' ou COLUMN = 'Concedido' e VALUE = 'N', caso o objetivo seja classificar a informação da coluna Concedido.
#### Caso queira adicionar um novo metodo classificador, lembrar de importá-lo (na primeira célula, onde há os 'imports') e adicioná-lo ao dicionário de classificadores (quinta célula)

In [1]:
import pandas as pd
import numpy as np

#Import dos classificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score
import matplotlib.pyplot as plt

In [2]:
#Nome do arquivo
fileName = "CreditoKnime.csv"

#Diretorio raiz
rootPath = "C:\\Users\\rogerio.silva\\Documents\\ufrj\\IA-2020\\Arquivos"

#Caminho completo do arquivo
filePath = f'{rootPath}\\{fileName}'

In [3]:
#Le arquivo csv
data = pd.read_csv(filePath, delimiter=";")
numericData = pd.get_dummies(data)
numericData

,Sexo,Casado,Dependentes,Educacao,Autonomo,Renda,RendaSocio,ValorEmprestimo,Prazo,Adimplente,AreaPropriedade,ValorEmprestimo_log,Concedido_N,Concedido_S
0,1,0,0,1,0,5849,0,14641,3600,10,2,49864,0,1
1,1,1,1,1,0,4583,15080,1280,3600,10,0,48520,1,0
2,1,1,0,1,1,3000,0,660,3600,10,2,41896,0,1
3,1,1,0,0,0,2583,23580,1200,3600,10,2,47874,0,1
4,1,0,0,1,0,6000,0,1410,3600,10,2,49487,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,1,0,2900,0,710,3600,10,0,42626,0,1
610,1,1,3,1,0,4106,0,400,1800,10,0,36888,0,1
611,1,1,1,1,0,8072,2400,2530,3600,10,2,553,0,1
612,1,1,2,1,0,7583,0,1870,3600,10,2,5231,0,1


In [4]:
COLUMN = 'Concedido'
VALUE = 'S'

columnValue = f'{COLUMN}_{VALUE}'
columnsToDrop = [col for col in numericData.columns if COLUMN in col]

In [5]:
DECISION_TREE = 'Decision Tree'
RANDOM_FOREST = 'Random Forest'
K_NEIGHBORS = 'k Neighbors'
MLP = "Multi-layer Perceptron"
ADA_BOOST = "AdaBoost"

CLASSIFIERS = {
    DECISION_TREE: DecisionTreeClassifier(),
    RANDOM_FOREST: RandomForestClassifier(),
    K_NEIGHBORS:KNeighborsClassifier(),
    MLP: MLPClassifier(),
    ADA_BOOST: AdaBoostClassifier()
}

In [6]:
x = numericData.drop(columnsToDrop, axis=1)
y = numericData[columnValue]

#Gera dados de teste e de treino
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=0) 

def getMetrics(classifierName):
    '''
        Retorna array com nome do classificador e metricas na seguinte ordem:
        [classificador, VN, FN, VP, FP, Medida-F, Acuracia, Precisao, Sensibilidade, Especificidade],
        
        sendo: 
        VN: Verdadeiro Negativo
        FN: Falso Negativo
        VP: Verdadeiro Positivo
        FP: Falso Positivo
        
    '''
    
    #Define classificador utilizado
    classifier = CLASSIFIERS[classifierName]
    classifier.fit(xTrain, yTrain)
    
    #Faz predicao
    yPred = classifier.predict(xTest)

    #Calcula matriz de confusao
    neg, falsePos, falseNeg, pos = confusion_matrix(yTest.to_numpy(),yPred).ravel() 
    
    #Calcula demais metricas
    fMeasure = f1_score(yTest, yPred)
    accuracy = accuracy_score(yTest, yPred)
    precision = precision_score(yTest, yPred)
    recall = recall_score(yTest, yPred) #recall tambem é chamado de sensibilidade
    specificity = neg /(neg + falsePos)
    
    return np.array([classifierName, neg, falseNeg, pos, falsePos, fMeasure, accuracy,precision, recall, specificity])


#Array com cabeçalho dos dados
metrics =[['classificador', 'VN', 'FN', 'VP', 'FP', 'Medida-F', 'Acuracia','Precisao', 'Sensibilidade', 'Especificidade']]

#Calcula metricas dos classificadores e adiciona na lista metrics
for classifierName in list(CLASSIFIERS.keys()):
    metrics = np.concatenate((metrics, [getMetrics(classifierName)]))

df2 = pd.DataFrame(metrics[1:], columns=metrics[0])
df2.to_csv(f'{rootPath}\\metricas.csv')
df2

,classificador,VN,FN,VP,FP,Medida-F,Acuracia,Precisao,Sensibilidade,Especificidade
0,Decision Tree,18,20,70,15,0.7999999999999999,0.7154471544715447,0.8235294117647058,0.7777777777777778,0.5454545454545454
1,Random Forest,15,5,85,18,0.8808290155440415,0.8130081300813008,0.8252427184466019,0.9444444444444444,0.45454545454545453
2,k Neighbors,5,13,77,28,0.7897435897435897,0.6666666666666666,0.7333333333333333,0.8555555555555555,0.15151515151515152
3,Multi-layer Perceptron,9,30,60,24,0.689655172413793,0.5609756097560976,0.7142857142857143,0.6666666666666666,0.2727272727272727
4,AdaBoost,15,7,83,18,0.869109947643979,0.7967479674796748,0.8217821782178217,0.9222222222222223,0.45454545454545453
